In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

import env
import wrangle

# acquire data 
df = wrangle.get_zillow_data()

In [2]:
# Take a look at the data

df.head(10)

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
0,11393337,2463969,NaN,NaN,NaN,3.0,3.0,NaN,4.0,3.0,...,43439.0,2016.0,22755.0,756.94,Y,14.0,6.037235e+13,117,0.086137,2017-06-08
1,11289917,2061546,1.0,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,136104.0,2016.0,27214.0,2319.90,Y,15.0,6.037901e+13,1248,-0.362001,2017-06-23
2,11705026,1834372,NaN,NaN,NaN,1.0,2.0,NaN,6.0,1.0,...,35606.0,2016.0,23624.0,543.69,None,NaN,6.037232e+13,1772,-0.146056,2017-06-30
3,14269464,1923117,NaN,NaN,NaN,3.0,4.0,NaN,NaN,3.0,...,880456.0,2016.0,445569.0,9819.72,None,NaN,6.059064e+13,2028,0.021085,2017-06-01
4,11389003,2121349,NaN,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,614000.0,2016.0,449000.0,7673.19,None,NaN,6.037703e+13,3273,-0.325393,2017-06-01
5,11967869,2093710,NaN,NaN,NaN,1.0,2.0,NaN,5.0,1.0,...,274237.0,2016.0,191821.0,3267.47,None,NaN,6.037185e+13,3429,-0.005566,2017-06-29
6,11696784,496818,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,168828.0,2016.0,127691.0,2135.39,None,NaN,6.037235e+13,3908,-0.124790,2017-06-16
7,14621355,505129,NaN,NaN,NaN,1.5,2.0,NaN,NaN,1.5,...,95442.0,2016.0,53625.0,998.18,None,NaN,6.059063e+13,4505,-0.256376,2017-06-27
8,17295416,2506407,NaN,NaN,NaN,2.0,2.0,NaN,NaN,2.0,...,239109.0,2016.0,59356.0,3209.98,None,NaN,6.111006e+13,4530,-0.189044,2017-05-16
9,12035176,1288537,NaN,NaN,NaN,1.0,1.0,NaN,11.0,1.0,...,245906.0,2016.0,146810.0,2926.19,None,NaN,6.037189e+13,5444,-0.114435,2017-05-24


In [3]:
# Looking at the columns with more detail

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 62 columns):
parcelid                        5000 non-null int64
id                              5000 non-null int64
airconditioningtypeid           1603 non-null float64
architecturalstyletypeid        13 non-null float64
basementsqft                    3 non-null float64
bathroomcnt                     4999 non-null float64
bedroomcnt                      4999 non-null float64
buildingclasstypeid             1 non-null float64
buildingqualitytypeid           3171 non-null float64
calculatedbathnbr               4955 non-null float64
decktypeid                      42 non-null float64
finishedfloor1squarefeet        380 non-null float64
calculatedfinishedsquarefeet    4986 non-null float64
finishedsquarefeet12            4770 non-null float64
finishedsquarefeet13            3 non-null float64
finishedsquarefeet15            180 non-null float64
finishedsquarefeet50            380 non-null flo

In [4]:
df = df.rename(columns={"bedroomcnt": "bedrooms", "bathroomcnt": "bathrooms", "calculatedfinishedsquarefeet": "square_feet", 
                        "taxamount": "taxes", "regionidzip": "zip", "taxvaluedollarcnt": "tax_value", "yearbuilt": "year_built"})

In [ ]:
def plot_variable_pairs(df):